# Comparison between OmniSci and Bodo

## Setup

First, you probably will need to download the NYC Yellow taxi data from 2015.
Uncomment the following code and change the **DATA_DIR** to your local directory, so you can run the cell.

In [ ]:
import time
import pandas as pd

In [ ]:
"""
%%bash

URL_ROOT=https://s3.amazonaws.com/nyc-tlc/trip+data
DATA_DIR=/work/bodoai/dataset/nyc-trip-2015

for i in {1..12}; do
    month=`printf "%2.0d\n" $i |sed "s/ /0/"`;
    FILENAME=yellow_tripdata_2015-${month}.csv;
    # wget -c ${URL_ROOT}/${FILENAME} -O ${DATA_DIR}/${FILENAME};
done
""";

As **bodo** **read_csv** works easily with constant paths than variable paths, you can link this folder inside our current path (optional, uncomment the code below if you want to run it).

In [ ]:
"""
%%bash

mkdir -p ./data
ln -s /work/bodoai/dataset/nyc-trip-2015 ./data
""";

In [ ]:
!ls ./data/nyc-trip-2015

As it is easier to work with `parquet` file than `csv` files, we will convert the `csv` files to a single `parquet` file (for now lets just use the first 3 files).

In [ ]:
dfs = []
data_dir = './data/nyc-trip-2015'

for i in range(1, 4):
    df = pd.read_csv(
        f'{data_dir}/yellow_tripdata_2015-{str(i).rjust(2, "0")}.csv',
        parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'],
    )
    dfs.append(df)
    del df

df = pd.concat(dfs)
df.to_parquet('./data/nyc-trip-2015/q1.pq')
del df

In [ ]:
!ls ./data/nyc-trip-2015

The next step now is to load this data to **OmniSci Database**. This document 
will not explain how to install and start up a **OmniSciDB** instance. For more information, 
check the [official documentation](https://docs.omnisci.com/installation-and-configuration/installation).

With the **OmniSciDB** running, lets create the table and load the data there.

In [ ]:
%%bash

source activate omniscidb

# echo "DROP TABLE IF EXISTS trips;" | omnisql -u admin -p HyperInteractive --port 6274
omnisql -u admin -p HyperInteractive --port 6274 < ../_setup/omnisci/schema_nyc2015.sql || true

In [ ]:
%%bash

source activate omniscidb

DATA_DIR=$(pwd)/data/nyc-trip-2015

for filename in ${DATA_DIR}/*.csv; do
  echo "COPY trips FROM '$filename' WITH (header='false');" | \
      omnisql -u admin -p HyperInteractive --port 6274
done

Now, it is ready to start, in the next sections we will compare the 
performance for some operations using **Bodo** and using **OmniSci**.

The functions that would be measured are: ...

- (f1) the average of `total_amount` by `passanger_count`
- (f2) the average of `total_amount` by `passenger_count` and `pickup_month`
- (f3) the count of trips grouped by `passenger_count`, `pickup_month` and `payment_type`

In [ ]:
funcs = {'f1': 0, 'f2': 0, 'f3': 0}

timelog = {
    'bodo': dict(funcs),
    'omni': dict(funcs),
}

## Bodo

In [ ]:
import bodo

In [ ]:
@bodo.jit
def read_data():
    df = pd.read_parquet('./data/nyc-trip-2015/q1.pq')
    return df


df = read_data()
df.info()

#### (f1) the average of `total_amount` by `passanger_count`

In [ ]:
@bodo.jit
def f1(df_nyc):
    return df_nyc.group_by('passenger_count').aggregate(
        df_nyc.total_amount.mean().name('_mean')
    )


f1(df)

#### (f2) the average of `total_amount` by `passenger_count` and `pickup_month`

In [ ]:
@bodo.jit
def f2(df_nyc):
    return df_nyc.group_by(['passenger_count']).aggregate(
        df_nyc.total_amount.mean().name('_mean')
    )


f2(df)

#### (f3) the count of trips grouped by `passenger_count`, `pickup_month` and `payment_type`

## OmniSci

Now, we will try the same operations using OmniSci <!-- with User-Defined Functions (UDFs)-->!
First, we need to install `pymapd`

In [ ]:
!pip install pymapd "pyarrow==0.17"

In [ ]:
import pymapd


con = pymapd.connect(
    user='admin', password='HyperInteractive',
    host='127.0.0.1', port=6274, dbname='omnisci'
)

#### (f1) the average of total_amount by passanger_count

In [ ]:
sql = '''
SELECT AVG(total_amount) 
FROM trips
GROUP BY passenger_count
'''
t0 = time.time()
result = con.execute(sql)
t1 = time.time()

timelog['omni']['f1'] = t1 - t0

for r in result:
    print(r)

#### (f2) the average of total_amount by passenger_count and pickup month

In [ ]:
sql = '''
SELECT AVG(total_amount) 
FROM (
    SELECT 
        total_amount, 
        passenger_count,
        EXTRACT(MONTH FROM tpep_pickup_datetime) AS pickup_month
    FROM trips
) AS _trips
GROUP BY passenger_count, pickup_month
'''
t0 = time.time()
result = con.execute(sql)
t1 = time.time()

timelog['omni']['f2'] = t1 - t0

for r in result:
    print(r)

#### (f3) the count of trips grouped by passenger_count, pickup month and payment type

In [ ]:
sql = '''
SELECT AVG(total_amount) 
FROM (
    SELECT 
        total_amount, 
        passenger_count,
        EXTRACT(MONTH FROM tpep_pickup_datetime) AS pickup_year,
        payment_type
    FROM trips
) AS _trips
GROUP BY passenger_count, pickup_year, payment_type
'''
t0 = time.time()
result = con.execute(sql)
t1 = time.time()

timelog['omni']['f3'] = t1 - t0

for r in result:
    print(r)

## Benchmark Result

In [ ]:
for tech_name, tech_result in timelog.items():
    print(tech_name)
    for f_name, f_time in tech_result.items():
        print(f'{f_name}: {f_time}')